In [1]:
import numpy as np
import torch

In [10]:
# fitur per kolom : (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                 [91, 88, 64],
                 [87, 134, 58],
                 [102, 43, 37],
                 [69, 96, 70]], dtype='float32')

# Target : (apel, jeruk)
targets = np.array([[56, 70],
                 [81, 101],
                 [119, 133],
                 [22, 37],
                 [103, 119]], dtype='float32')

Fitur dan target harus dipisah, karena cara pengolahannya berbeda.
Disini pakai numpy, karena untuk mentraining data, biasanya pakai numpy

In [11]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


selanjutnya untuk menentukan weight dan bias untuk model linear regression, pakai angka random dulu.
 w dan b memiliki bentuk seperti matrix. jumlah kolom w mengikuti jumlah fitur, jumlah kolom b mengikuti jumlah kelas. Matrix w nantinya akan di tranpose dan dikalikan dengan matrix fitur/input

In [12]:
#weight dan bias
w = torch.randn(2,3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-0.5898,  1.5420, -1.3222],
        [-0.6676,  0.0367,  1.6924]], requires_grad=True)
tensor([-0.9665, -0.2799], requires_grad=True)


In [14]:
def model(x) :
    return x @ w.t()+b

x adalah input, @ berarti perkalian matrix yang sesuai, t berarti tranpose

In [15]:
# cara membuat prediksi pakai linear regression
preds = model(inputs)
print(preds)

tensor([[  2.4356,  26.2235],
        [ -3.5658,  50.5199],
        [ 77.6606,  44.7256],
        [-43.7448,  -4.1727],
        [ 13.8132,  75.6552]], grad_fn=<AddBackward0>)


In [16]:
# bandingkan dengan target yang asli
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


ada selisih yang sangat signifikan terhadap prediksi dan aktualnya. Ini dikarenakan diawal, menentukan w dan b secara random.
Percobaan pertama memang tidak selalu berhasil

## Loss Function
Sebelum memperbaiki model yang buruk tadi, ada namanya Loss Function. 
Loss Function adalah perbandingan hasil prediksi dengan target aktual. 
Hal ini berfungsi untuk menguji performa model yang digunakan.
Biasanya digunakan Mean Squared Error (MSE) sebagai nilai performa.
Rumus MSE = jumlah elemen data (selisih tiap elemen^2)/jumlah data

In [18]:
# MSE LOSS
def mse(t1,t2) :
    diff = t1-t2
    return torch.sum(diff*diff)/diff.numel()

In [28]:
# hitung loss
loss = mse(preds, targets)
print(loss)

tensor(3983.7097, grad_fn=<DivBackward0>)


cara bacanya gmn?
caranya adalah hasil perhitungannya di akar kuadrat. lalu akan muncul sebuah angka
angka tersebut dibandingkan dengan range target aktual.
 Hasil sqrt dari 3983 adalah 63. sedangkan range target aktual adalah 20-150

## Hitung gradient
Kalau pakai pytorch, bisa otomatis menghitung gradient atau turunan dari loss w.r.t hingga bisa dapat weight dan bias. Asalkan di set requires_grad = True.

In [ ]:
# hitung turunan
loss.backward()

In [24]:
# gradient dari weight
print(w)
print(w.grad)

tensor([[-0.5898,  1.5420, -1.3222],
        [-0.6676,  0.0367,  1.6924]], requires_grad=True)
tensor([[-5612.4170, -5591.8115, -3757.7612],
        [-4531.9297, -5027.1133, -2958.1106]])


In [25]:
# gradient dari bias
print(b)
print(b.grad)

tensor([-0.9665, -0.2799], requires_grad=True)
tensor([-66.8803, -53.4097])


kalau element gradient positif : naikkin nilai element akan sedikit naikkin loss
kalau element gradient negatif : naikkin nilai element akan sedikit turunin loss

Sebelum memperbaiki model, reset gradient terlebih dahulu.
karena fungsi backward hanya bisa sekali dijalankan

In [30]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


ingat, ini diawali dengan random number, jadi hasilnya memang acak. gabisa jadi panutan. Tapi asalkan modelnya diimprove dengan benar. Nantinya akan didapatkan model yang lebih baik

## Atur kembali weight dan bias pakai gradient turunan
- keluarkan hasil prediksi
- hitung loss
- hitung gradient w.r.t, weight, dan bias
- atur ulang weight dengan sedikit nilai
- reset gradient ke 0

In [32]:
#keluarkan hasil prediksi
preds = model(inputs)
print(preds)

tensor([[  2.4356,  26.2235],
        [ -3.5658,  50.5199],
        [ 77.6606,  44.7256],
        [-43.7448,  -4.1727],
        [ 13.8132,  75.6552]], grad_fn=<AddBackward0>)


In [33]:
# hitung loss
loss = mse(preds,targets)
print(loss)

tensor(3983.7097, grad_fn=<DivBackward0>)


In [34]:
# hitung gradient wrt, weight, dan bias
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-5612.4170, -5591.8115, -3757.7612],
        [-4531.9297, -5027.1133, -2958.1106]])
tensor([-66.8803, -53.4097])


In [36]:
# atur ulang weight dengan sedikit nilai
with torch.no_grad() :
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    
#     reset gradient ke 0
    w.grad.zero_()
    b.grad.zero_()

fungsi no_grad adalah untuk menyuruh pytorch untuk tidak mengubah gradient ketika sedang mengubah weight dan bias

In [39]:
# intip weight dan bias yang baru
print(w)
print(b)

tensor([[-0.5337,  1.5980, -1.2847],
        [-0.6223,  0.0870,  1.7220]], requires_grad=True)
tensor([-0.9659, -0.2794], requires_grad=True)


In [40]:
# cek loss baru
preds = model(inputs)
loss = mse(preds,targets)
print(loss)

tensor(2786.7432, grad_fn=<DivBackward0>)


nilai loss turun banyak, hanya dengan cara mengubah sedikit nilai weight dan bias

## Melakukan training regressi linear dengan beberapa epoch (perulangan)
untuk mengurangi nilai Loss lebih banyak, lakukan training berkali-kali
jangan lupa cek loss tiap perulangannya juga

In [41]:
# train 100 epoch
for i in range(100) :
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

cek kembali nilai loss setelah 100 kali dilakukan training

In [42]:
# hitung nilai loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(223.8214, grad_fn=<DivBackward0>)


bandingkan prediksi dengan aktual

In [43]:
preds

tensor([[ 56.6589,  70.1346],
        [ 68.2300, 107.7869],
        [151.2918, 117.0523],
        [ 17.7002,  37.0006],
        [ 79.3887, 131.4724]], grad_fn=<AddBackward0>)

In [44]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

hasilnya sudah cukup mendekati, hasil yang lebih baik bisa didapatkan apabila melakukan training beberapa epoch kembali

In [48]:
# train 50 epoch
for i in range(50) :
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [49]:
preds = model(inputs)
loss = mse(preds, targets)
print(loss)
print(preds)
print(targets)

tensor(180.9225, grad_fn=<DivBackward0>)
tensor([[ 57.3168,  69.9206],
        [ 69.7924, 107.0732],
        [146.6807, 119.0196],
        [ 21.6355,  35.7001],
        [ 79.8106, 130.9895]], grad_fn=<AddBackward0>)
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


bisa tambah epoch untuk hasil yang lebih bagus lagi